In [1]:
#needed for images in notes below
from IPython import display  
import string

# Encoding Words
We will process movie reviews that are labeled as having either positive or negative sentiment. This is a binary clasification problem.

https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data

In [2]:
import numpy as np
import tensorflow as tf
imdb = tf.keras.datasets.imdb

In [3]:
#load the data, but ony the first 10000 most popular words
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
#The warning that comes from this is tensorflow's fault not yours.
# Presumably a future version would fix it.

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\sbing\.conda\envs\Tensorflow\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\sbing\.conda\envs\Tensorflow\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you m

In [4]:
#get the dictionary that allows you to get a word's index
word_idxs = imdb.get_word_index()

In [5]:
#function to encode reviews as one-hot
def encode_review(review, dim):
    #create an array of 0s of size passed in
    review_vec = np.zeros(dim)
    #for each word in review (recall words are numbers)
    for word in review:
        #set the index for that word number to true
        review_vec[word] = 1
    #return completed encode
    return review_vec

In [6]:
#number of unique words
size = len(word_idxs.keys())
#encoded training data
one_hot_train = []
for rev in x_train:
    one_hot_train.append(encode_review(set(rev), size))
#inspect first 2 vals
one_hot_train[:2]

[array([0., 1., 1., ..., 0., 0., 0.]), array([0., 1., 1., ..., 0., 0., 0.])]

In [7]:
#always do the same thing you did to your train to your test data.
one_hot_test = []
for rev in x_test:
    one_hot_test.append(encode_review(set(rev), size))
one_hot_test = np.array(one_hot_test)

In [8]:
#convert ot numpy array
one_hot_train = np.array(one_hot_train)

## Now we can use the same basic network as last time, but we need more input nodes based on the shape

In [9]:
size = one_hot_train.shape[1]
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(size,)),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(32),
  #tf.keras.layers.Dropout(.2),  
  tf.keras.layers.Dense(16),
  tf.keras.layers.Dense(10, activation = 'sigmoid'),
  tf.keras.layers.Dense(1) #true=positive, false=negative
])

In [10]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [11]:
#batch size will take 64 pieces of my data at a time and run
# each batch through the network. If your computer is faster than mine
# you can probably bump this number up to 512 instead of 64

model.fit(one_hot_train, y_train, epochs=2, batch_size=512)

Epoch 1/2
49/49 [==============================] - 8s 161ms/step - loss: 0.5369 - accuracy: 0.7635
Epoch 2/2
49/49 [==============================] - 7s 150ms/step - loss: 0.2788 - accuracy: 0.9099


In [12]:
model.evaluate(one_hot_test,  y_test, verbose=2)

782/782 - 6s - loss: 0.4182 - accuracy: 0.8693


[0.41816651821136475, 0.8692799806594849]

In [13]:
def clean_word(word):
    for p in string.punctuation:
        word = word.replace(p, "")
    return word.lower() #return the word as lowercase

In [14]:
def predict(review):
    
    #review cannot have upper case or punctuation
    user_review = review
    #take each word out of the string
    user_review = user_review.split()
    clean_review = []
    for word in user_review:
        clean_review.append(clean_word(word))
    #create an empty list to hold the numbers
    print("clean_review: ", clean_review)
    ur = []
    for word in clean_review:
        #look up the word's index and append it to the list
        ur.append(word_idxs[word])
    #one-hot the review
    oh_ur = np.array(encode_review(set(ur), size))
    
    #review needs to be in a numpy outer array to be accepted by the model
    # needs shape (1,88584), or if there are more reviews it would be
    # more than 1 for first dimension

    oh_ur = np.array([oh_ur])
    predictions = model(oh_ur)
    print(predictions)
    if predictions > 0.5:
        return("Favorable")
    else: 
        return("Unfavorable")

## Make predictions

In [15]:
predict("I did not enjoy that Star Wars film.")

clean_review:  ['i', 'did', 'not', 'enjoy', 'that', 'star', 'wars', 'film']
tf.Tensor([[0.24734178]], shape=(1, 1), dtype=float32)


'Unfavorable'

In [16]:
predict("miss this movie i do not like it")

clean_review:  ['miss', 'this', 'movie', 'i', 'do', 'not', 'like', 'it']
tf.Tensor([[0.16972463]], shape=(1, 1), dtype=float32)


'Unfavorable'

In [17]:
predict("do not miss this movie i like it")

clean_review:  ['do', 'not', 'miss', 'this', 'movie', 'i', 'like', 'it']
tf.Tensor([[0.16972463]], shape=(1, 1), dtype=float32)


'Unfavorable'

In [18]:
predict("I fell asleep after the first scene")

clean_review:  ['i', 'fell', 'asleep', 'after', 'the', 'first', 'scene']
tf.Tensor([[0.24452156]], shape=(1, 1), dtype=float32)


'Unfavorable'

In [19]:
predict("I thought that the special effects were magnificient")

clean_review:  ['i', 'thought', 'that', 'the', 'special', 'effects', 'were', 'magnificient']
tf.Tensor([[0.27021098]], shape=(1, 1), dtype=float32)


'Unfavorable'

## Homework Due Mon. before 10PM

1. Modify the network above to be deeper with more layers. Use the ideas we talked about last time to shrink layer size as you decrease. Add dropout and activations as well. Based on the answer we want and your reading about sigmoid vs softmax, choose the correct activation for the final layer.

2. Reconfigure the cell that formats a string into a review and make it a function def that can be called multiple times.

3. Make a new cell that passes a few negative and positive sentences that you write, into your new function. Collect all the encoded sentences into one array.

4. Make predictions about all of your encoded sentences in your array and print the predictions out to see if they are correct (only one predictions array should need to be created by the model).

5. Try out the following two sentences (remember no punctuation or upper case):
    - "miss this movie i do not like it"
    - "do not miss this movie i like it"
    
    Why do you think they yield the same result?

_Next time we will continue with words and sentiments and fix the issue  part 5 reveals._